## Analyze whether SNWD varies more from year to year or from place to place.

In [2]:
import pandas as pd
import numpy as np
import urllib
import math

In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [5]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBBBBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
564
+------------------+-------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|           coeff_1|            coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|               res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+------------------+-------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|13523.756321247925|-3777.0542099732156|1835.5544645016141|   3448.2|SSSBBBBB|    39.8|-105.7833|       SNWD|0.09913711457955644|0.03026537338607403|0.015850354815698863|0.7631

In [6]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [7]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


station    year      coeff_2
0  USC00050674  1972.0 -3777.054210
1  USC00051660  1975.0 -2692.581852
2  USC00050674  1967.0 -3320.445981
3  USC00051660  1993.0 -3850.707795
4  USC00051660  1968.0 -3000.264089

In [8]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station  US1COEG0008  US1COGL0013  US1COJF0024  US1COJF0040  US1COJF0051  \
year                                                                       
1911.0           NaN          NaN          NaN          NaN          NaN   
1916.0           NaN          NaN          NaN          NaN          NaN   
1917.0           NaN          NaN          NaN          NaN          NaN   
1918.0           NaN          NaN          NaN          NaN          NaN   
1919.0           NaN          NaN          NaN          NaN          NaN   
1932.0           NaN          NaN          NaN          NaN          NaN   
1933.0           NaN          NaN          NaN          NaN          NaN   
1948.0           NaN          NaN          NaN          NaN          NaN   
1949.0           NaN          NaN          NaN          NaN          NaN   
1950.0           NaN          NaN          NaN          NaN          NaN   

station  US1COJF0066  US1COJF0094  US1COJF0104  US1COJF0106  US1COJF0218  \
year                                                                       
1911.0           NaN          NaN          NaN          NaN          NaN   
1916.0           NaN          NaN          NaN          NaN          NaN   
1917.0           NaN          NaN          NaN          NaN          NaN   
1918.0           NaN          NaN          NaN          NaN          NaN   
1919.0           NaN          NaN          NaN          NaN          NaN   
1932.0           NaN          NaN          NaN          NaN          NaN   
1933.0           NaN          NaN          NaN          NaN          NaN   
1948.0           NaN          NaN          NaN          NaN          NaN   
1949.0           NaN          NaN          NaN          NaN          NaN   
1950.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00056970  USC00057031  USC00057033  USC00057618  \
year        ...                                                            
1911.0      ...               NaN          NaN          NaN          NaN   
1916.0      ...               NaN   367.267198          NaN          NaN   
1917.0      ...               NaN          NaN          NaN          NaN   
1918.0      ...               NaN          NaN          NaN          NaN   
1919.0      ...               NaN          NaN          NaN          NaN   
1932.0      ...               NaN   516.402597          NaN          NaN   
1933.0      ...               NaN   430.415685          NaN          NaN   
1948.0      ...               NaN          NaN          NaN          NaN   
1949.0      ...               NaN   452.409868          NaN  2515.322459   
1950.0      ...               NaN          NaN          NaN  1052.945721   

station  USC00058022  USC00058064  USC00058212  USC00058575  USW00023063  \
year                                                                       
1911.0           NaN          NaN          NaN          NaN          NaN   
1916.0           NaN          NaN          NaN          NaN          NaN   
1917.0           NaN          NaN          NaN          NaN          NaN   
1918.0           NaN          NaN          NaN          NaN          NaN   
1919.0           NaN          NaN          NaN          NaN          NaN   
1932.0           NaN          NaN          NaN          NaN          NaN   
1933.0           NaN          NaN          NaN          NaN          NaN   
1948.0           NaN          NaN          NaN          NaN          NaN   
1949.0           NaN          NaN  2279.684195          NaN  1499.424067   
1950.0           NaN          NaN   662.099816          NaN          NaN   

station  USW00093009  
year                  
1911.0           NaN  
1916.0           NaN  
1917.0           NaN  
1918.0           NaN  
1919.0           NaN  
1932.0           NaN  
1933.0           NaN  
1948.0           NaN  
1949.0   1099.035879  
1950.0     29.185208  

[10 rows x 56 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [9]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  1476.9291627
RMS removing mean-by-station=  925.75164404
RMS removing mean-by-year   =  1248.39893937


In [10]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 1476.9291627
0 after removing mean by year    = 1248.39893937
0 after removing mean by stations= 769.487921335
1 after removing mean by year    = 736.288083215
1 after removing mean by stations= 730.237782325
2 after removing mean by year    = 728.193183122
2 after removing mean by stations= 727.315384544
3 after removing mean by year    = 726.903666233
3 after removing mean by stations= 726.702777377
4 after removing mean by year    = 726.60258485
4 after removing mean by stations= 726.551893039
